In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import MSELoss, CrossEntropyLoss
from torchvision import transforms
import torch.optim as optim
import os
import time

from data_loader import FaceDataset
from model import AgeGenderModel
torch.cuda.is_available()

True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

t = torch.tensor([1,2], device=device)
device
t = t*2

In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])  

full_dataset = FaceDataset(root_dir='part1', transform=transform)

In [5]:
filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
dataset = torch.utils.data.Subset(full_dataset, filtered_indices)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


In [6]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 4)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 4)

In [7]:
image_files = [f for f in os.listdir('part1') if f.endswith(".jpg")]
len(image_files)
for i in image_files:
    parts = i.split('_')
    gender = int(parts[1])
    if gender > 1:
        print(gender, i)

In [8]:
next(iter(train_loader))[1].shape
next(iter(train_loader))[1][:, 1]
z = 1
for i, (images, labels) in enumerate(train_loader) :
    images = images.to(device)
    ages = labels[:, 0].to(device)
    genders = labels[:, 1].long().to(device)
    print(genders)
    z += 1
    if z > 5: break

tensor([0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 1, 0], device='cuda:0')
tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 1], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')
tensor([1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 0], device='cuda:0')
tensor([1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0], device='cuda:0')


In [9]:
model = AgeGenderModel()
print(next(model.parameters()).dtype)

torch.float32


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AgeGenderModel().to(device)

age_criterion = MSELoss()
gender_criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [11]:
num_gender_classes = 2

num_epochs = 20
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader) :
        images = images.to(device)
        ages = labels[:, 0].to(device)
        genders = labels[:, 1].long().to(device)
        max_gender = genders.max().item()
        if max_gender >= num_gender_classes:
            print(i, genders)
            print(f"Error in batch {i} Epoch {epoch+1}")
            print(f"Maximun gender value {max_gender}")
            break

        optimizer.zero_grad()

        age_pred, gender_pred = model(images)

        age_loss = age_criterion(age_pred.squeeze(), ages)
        gender_loss = gender_criterion(gender_pred, genders)
        total_loss = age_loss + gender_loss

        total_loss.backward()

        optimizer.step()

        running_loss += total_loss.item()
    end_time = time.time()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):4f}, Time: {end_time - start_time:.2f}sec")

torch.save(model.state_dict(),'models/age_gender_model.pth')
print("Training complete. Model saved.")


Epoch 1/20, Loss: 309.706742, Time: 17.55sec
Epoch 2/20, Loss: 145.057676, Time: 17.38sec
Epoch 3/20, Loss: 115.555329, Time: 17.42sec
Epoch 4/20, Loss: 84.926439, Time: 17.67sec
Epoch 5/20, Loss: 75.874583, Time: 17.59sec
Epoch 6/20, Loss: 60.225246, Time: 17.64sec
Epoch 7/20, Loss: 53.498460, Time: 17.51sec
Epoch 8/20, Loss: 52.052705, Time: 17.52sec
Epoch 9/20, Loss: 50.825881, Time: 17.54sec
Epoch 10/20, Loss: 34.590564, Time: 17.65sec
Epoch 11/20, Loss: 28.488644, Time: 17.62sec
Epoch 12/20, Loss: 31.371851, Time: 17.58sec
Epoch 13/20, Loss: 20.677656, Time: 17.63sec
Epoch 14/20, Loss: 15.551998, Time: 17.70sec
Epoch 15/20, Loss: 13.550428, Time: 17.61sec
Epoch 16/20, Loss: 10.885607, Time: 17.62sec
Epoch 17/20, Loss: 10.209280, Time: 17.89sec
Epoch 18/20, Loss: 8.080401, Time: 17.85sec
Epoch 19/20, Loss: 10.020575, Time: 17.59sec
Epoch 20/20, Loss: 10.109992, Time: 17.55sec
Training complete. Model saved.


In [26]:
import torch
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix
import numpy as np

from data_loader import FaceDataset
from model import AgeGenderModel

In [ ]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
root_dir = '/Users/vaishnavishinde/Desktop/age_detector/dataset'
device = torch.device("cpu")

# Load the trained model
model = AgeGenderModel().to(device)
model.load_state_dict(torch.load('age_gender_model.path'))
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'age_gender_model.path'

In [ ]:
full_dataset = UTKFaceDataset(root_dir=root_dir, transform=transform)
filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
dataset = torch.utils.data.Subset(full_dataset, filtered_indices)

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
_, _, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:


def main():
    root_dir = 'path/to/UTKFace'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load the trained model
    model = AgeGenderModel().to(device)
    model.load_state_dict(torch.load('age_gender_model.pth'))
    model.eval()
    
    # Recreate the dataset split for a consistent test set
    full_dataset = UTKFaceDataset(root_dir=root_dir, transform=get_transforms())
    filtered_indices = [i for i, (data, label) in enumerate(full_dataset) if data is not None]
    dataset = torch.utils.data.Subset(full_dataset, filtered_indices)
    
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    _, _, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_ages = []
    all_age_preds = []
    all_genders = []
    all_gender_preds = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            ages = labels[:, 0].numpy()
            genders = labels[:, 1].numpy()
            
            age_pred, gender_pred = model(images)
            
            all_ages.extend(ages)
            all_genders.extend(genders)
            all_age_preds.extend(age_pred.squeeze().cpu().numpy())
            all_gender_preds.extend(torch.argmax(gender_pred, dim=1).cpu().numpy())

    # Gender metrics
    gender_accuracy = accuracy_score(all_genders, all_gender_preds)
    conf_matrix = confusion_matrix(all_genders, all_gender_preds)

    # Age metrics
    mae = mean_absolute_error(all_ages, all_age_preds)
    
    print(f"Gender Accuracy: {gender_accuracy:.4f}")
    print("Gender Confusion Matrix:\n", conf_matrix)
    print(f"Age Mean Absolute Error (MAE): {mae:.2f}")

if __name__ == '__main__':
    main()